In [188]:
%run web_scrape_func_def.ipynb

imports loaded.

> file management func:

    get_content(directory)
    get_last_update(file_path)

> fighters_stats_scrape func:

    get_xml(url, headers = {'User-Agent':'Moz...}) # Return web page data in .xml format.
    get_links(url)
    get_fighter_data(url) # Retrive data as dictionery
    get_fighter_data(url) # Retrive data as list
    get_current_date()

> matches_stats_web_scrape func:

>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>
    get_event_data(soup)
    extract_fight_data(event_url)
    get_fight_columns(soup)
    get_fight_list(soup)
    get_event_date(soup)
    get_later_date(date1, date2)


# Main( )

### Extracting links:

In [148]:
links = []
for page in tqdm(ascii_letters.lower()):
    url = 'http://ufcstats.com/statistics/fighters?char=' + page + '&page=all'
    links.extend(get_links(url=url))
    
links = np.unique(links)
print('array length: %d' %len(links))
links[len(links)-4:len(links)-1]

100%|██████████| 52/52 [00:37<00:00,  1.39it/s]

array length: 3951


array(['http://ufcstats.com/fighter-details/ffc3e6daaa6da0b7',
       'http://ufcstats.com/fighter-details/ffdeb4fbea09ce75',
       'http://ufcstats.com/fighter-details/ffe9703408fb5964'],
      dtype='<U52')

### Stacking all fighters data (using multiprocessing):

In [183]:
# multiprocessing
p = multiprocessing.cpu_count() - 2

# Create a pool of worker processes.
pool = Pool(processes=p)
batch_size = 100
data = []

for i in tqdm(range(0,len(links),batch_size)):
    
    # Load fighter data from each link in parallel.
    row = pool.map(get_fighter_data, links[i:i + batch_size])
    batch_data = np.vstack(row)
    data.extend(batch_data)
    #data = np.vstack(row)
    
# Print the resulting data set.
print(data[0:5])

100%|██████████| 40/40 [06:36<00:00,  9.91s/it]

[array(['Gabriel Silva', '8', '2', '0', '5\'6"', '135lbs.', '71"',
       'Orthodox', 'Aug26,1994', '1.30', '34%', '3.47', '45%', '1.50',
       '25%', '38%', '0.5', '27-03-2023', '29-02-2020'], dtype=object), array(['Aalon Cruz', '8', '4', '0', '6\'0"', '145lbs.', '78"', 'Switch',
       'Sep20,1989', '7.58', '39%', '8.88', '58%', '0.00', '0%', '0%',
       '0.0', '27-03-2023', '06-03-2021'], dtype=object), array(['Davi Ramos', '10', '4', '0', '5\'6"', '155lbs.', '70"',
       'Orthodox', 'Nov05,1986', '2.64', '37%', '3.44', '56%', '1.95',
       '40%', '83%', '0.6', '27-03-2023', '18-07-2020'], dtype=object), array(['Tyler Hill', '8', '2', '0', '6\'3"', '170lbs.', '--', 'Switch',
       'Aug10,1991', '3.64', '39%', '5.33', '47%', '0.00', '0%', '0%',
       '0.0', '27-03-2023', '19-06-2018'], dtype=object), array(['Wesley Little', '4', '8', '0', '6\'4"', '205lbs.', '--', '', '--',
       '0.00', '0%', '0.00', '0%', '0.00', '0%', '0%', '0.0',
       '27-03-2023', '19-11-2010'], dtype=o

In [185]:
columns = np.array(['name','wins','losses','draws','height','weight','reach','stance','dob','slpm','str_acc','sapm','str_def','td_avg','td_acc','td_def','sub_avg','last_update','last_match'])
df = pd.DataFrame(data=data, columns=columns)
df.tail(5)

,name,wins,losses,draws,height,weight,reach,stance,dob,slpm,str_acc,sapm,str_def,td_avg,td_acc,td_def,sub_avg,last_update,last_match
3946,Quinton Jackson,37,12,0,"6'1""",205lbs.,"73""",Orthodox,"Jun20,1978",3.25,54%,2.51,55%,1.52,56%,73%,0.1,27-03-2023,25-04-2015
3947,Johnny Bedford,21,13,1 (1 NC),"5'10""",135lbs.,"71""",Orthodox,"Jan06,1983",4.53,50%,1.75,63%,1.83,62%,62%,0.0,27-03-2023,13-09-2014
3948,Jessica Rakoczy,1,5,0 (1 NC),"5'7""",115lbs.,--,Orthodox,"Apr14,1977",2.50,44%,3.55,64%,0.75,20%,0%,0.8,27-03-2023,25-04-2015
3949,David Onama,10,2,0,"5'11""",145lbs.,"74""",Orthodox,"Jun07,1994",5.30,45%,5.75,51%,1.06,20%,45%,0.3,27-03-2023,13-08-2022
3950,Dillon Smith,0,0,0,"6'0""",170lbs.,--,,--,0.00,0%,0.00,0%,0.00,0%,0%,0.0,27-03-2023,25-09-2009


### Saving file:

In [186]:
loc = '../data/'
df.to_csv('../data/ufc_fighters_stats.csv', index=False)
print('../data/ufc_fighters_stats.csv saved.')        

../data/ufc_fighters_stats.csv saved.
